The **Sequential** model is a linear stack of layers.
Create a Sequential model by passing a list of layer instances to the constructor.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
        Dense(32, input_dim=784),
        Activation('relu'),
        Dense(10),
        Activation('softmax'),
    ])

Using TensorFlow backend.


Alternatively, layers can be added via the .add() method.

In [2]:
model = Sequential()
model.add(Dense(32, input_dim=784))
model.add(Activation('relu'))

Specifying the input shape. All three cells below are equivalent.

In [3]:
# pass an input_shape argument to the first layer. 
# This is a shape tuple (a tuple of integers or None entries, where None indicates 
# that any positive integer may be expected). In input_shape, the batch dimension is not included.
model = Sequential()
model.add(Dense(32, input_shape=(784,)))

In [4]:
# pass instead a batch_input_shape argument, where the batch dimension is included. 
# This is useful for specifying a fixed batch size (e.g. with stateful RNNs).
model = Sequential()
model.add(Dense(32, batch_input_shape=(None, 784)))
# Note that the batch dimension is "None" here.
# So, the model will be able to process batches of any size.

In [5]:
# some 2D layers, such as Dense, support the specification of their input shape via the argument input_dim, 
# and some 3D temporal layers support the arguments input_dim and input_length.
model = Sequential()
model.add(Dense(32, input_dim=784))

All three cells below are equivalent.

In [6]:
from keras.layers import LSTM
model = Sequential()
model.add(LSTM(32, input_shape=(10, 64)))

In [7]:
model = Sequential()
model.add(LSTM(32, batch_input_shape=(None, 10, 64)))

In [8]:
model = Sequential()
model.add(LSTM(32, input_length=10, input_dim=64))

**The Merge Layer**

In [9]:
from keras.layers import Merge

left_branch = Sequential()
left_branch.add(Dense(32, input_dim=784))

right_branch = Sequential()
right_branch.add(Dense(32, input_dim=784))

merged = Merge([left_branch, right_branch], mode='concat')

final_model = Sequential()
final_model.add(merged)
final_model.add(Dense(10, activation='softmax'))

In [10]:
# An example showing how the above two-branch model can be trained
final_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
#final_model.fit([input_data_1, input_data_2], targets) # We pass one data array per model input

In [11]:
# We can also pass a function as the mode argument (allowing for arbitrary transformations)
#merged = Merge([left_branch, right_branch], mode=lambda x: x[0] - x[1])

Model configuration

In [12]:
# Before training a model, the learning process needs to be configured via the compile method, using three arguments.
# We need an optimizer, a loss function, and a list of metrics

# for a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# for a binary classification problem
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# for a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse')

# for custom metrics
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

'''def false_rates(y_true, y_pred):
    false_neg = ...
    false_pos = ...
    return {
        'false_neg': false_neg,
        'false_pos': false_pos,
    }'''

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', mean_pred]) #, false_rates])

**Training**

In [13]:
# Keras models are trained on Numpy arrays of input data and labels. 
# For training a model, we typically use the fit function.

# for a single-input model with two classes (binary):

model = Sequential()
model.add(Dense(1, input_dim=784, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# generate dummy data
import numpy as np
data = np.random.random((1000, 784))
labels = np.random.randint(2, size=(1000, 1))

# train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, nb_epoch=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 0.7279 - acc: 0.4940     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 0.7146 - acc: 0.5100     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 0.7038 - acc: 0.5190     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 0.6940 - acc: 0.5370     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 0.6948 - acc: 0.5590     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 0.6775 - acc: 0.5700     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 0.6828 - acc: 0.5860     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 0.6666 - acc: 0.6040     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 0.6630 - acc: 0.6110     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 0.6554 - acc: 0.6300     


In [14]:
# for a multi-input model with 10 classes:

left_branch = Sequential()
left_branch.add(Dense(32, input_dim=784))

right_branch = Sequential()
right_branch.add(Dense(32, input_dim=784))

merged = Merge([left_branch, right_branch], mode='concat')

model = Sequential()
model.add(merged)
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# generate dummy data
import numpy as np
from keras.utils.np_utils import to_categorical
data_1 = np.random.random((1000, 784))
data_2 = np.random.random((1000, 784))

# these are integers between 0 and 9
labels = np.random.randint(10, size=(1000, 1))
# we convert the labels to a binary matrix of size (1000, 10)
# for use with categorical crossentropy
labels = to_categorical(labels, 10)

# train the model
# note that we are passing a list of Numpy arrays as training data since the model has 2 inputs
model.fit([data_1, data_2], labels, nb_epoch=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 2.9087 - acc: 0.1050     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 2.5968 - acc: 0.1310     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 2.4351 - acc: 0.1600     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 2.3888 - acc: 0.1670     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 2.2410 - acc: 0.2040     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 2.1614 - acc: 0.2270     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 2.0176 - acc: 0.2850     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 1.9037 - acc: 0.3360     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 1.9081 - acc: 0.3360     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 1.7781 - acc: 0.4110     
